La tabla PGA contiene los indicadores **Physician´s Global Assessment** que se toman a cada paciente en cada cita.

Resultan de una evaluación subjetiva de la gravedad de la enfermedad por parte del paciente.


In [1]:
import pandas as pd
import numpy as np
df_PGA = pd.read_excel("Psoriasis/PGA.xlsx")
print(df_PGA.columns)


Index(['ID', 'Fecha', 'Resultado Total', 'Pregunta', 'Respuesta',
       'Sede de atención'],
      dtype='object')


In [2]:
df_PGA.rename(columns={"Resultado Total": "Total"}, inplace = True)

df_PGA['Total'] = pd.to_numeric(df_PGA["Total"],errors='coerce')
df_PGA

,ID,Fecha,Total,Pregunta,Respuesta,Sede de atención
0,1347761.0,2018-05-15 08:56:44,3,Eritema,3.0,NaN
1,1347761.0,2018-05-15 08:56:44,3,Endurecimiento,3.0,NaN
2,1347761.0,2018-05-15 08:56:44,3,Descamación,3.0,NaN
3,2910702.0,2018-05-23 13:37:42,1,Eritema,1.0,NaN
4,2910702.0,2018-05-23 13:37:42,1,Endurecimiento,1.0,NaN
...,...,...,...,...,...,...
8551,2149752.0,2020-05-27 13:33:12,0,Endurecimiento,0.0,Almacentro Piso 11
8552,2149752.0,2020-05-27 13:33:12,0,Descamación,0.0,Almacentro Piso 11
8553,1195834.0,2020-05-28 14:45:31,2,Eritema,2.0,Almacentro Piso 11
8554,1195834.0,2020-05-28 14:45:31,2,Endurecimiento,2.0,Almacentro Piso 11


In [3]:
df_PGA_v2 = df_PGA.groupby(["ID","Fecha"]).mean()

df_PGA_v2.reset_index(inplace=True)
df_PGA_v2

,ID,Fecha,Total,Respuesta
0,502369.0,2019-04-24 07:54:55,0,0.000000
1,502369.0,2019-09-16 13:38:46,2,2.333333
2,502369.0,2020-01-29 09:03:42,0,0.000000
3,506165.0,2019-09-16 13:44:15,1,1.000000
4,510859.0,2019-08-12 14:28:20,3,3.333333
...,...,...,...,...
2834,3499636.0,2019-07-05 09:08:19,0,0.000000
2835,3499636.0,2019-10-30 13:59:15,0,0.000000
2836,3499636.0,2020-02-26 11:26:13,0,0.000000
2837,3499725.0,2019-05-24 12:48:58,0,0.000000


In [4]:
preguntas = pd.unique(df_PGA['Pregunta']).tolist()
indices = df_PGA_v2.index
df_PGA_v2["TOTAL"] = 0 
for Preguntak in preguntas:
        df_PGA_v2[Preguntak] = 0
        for ind_v2 in indices:
            ID = df_PGA_v2["ID"][ind_v2]
            Fecha = df_PGA_v2["Fecha"][ind_v2]
            ind_v1 = df_PGA[(df_PGA["ID"]==ID) & (df_PGA["Fecha"]==Fecha)  & (df_PGA["Pregunta"]==Preguntak)].index.tolist()
            if ind_v1:
                df_PGA_v2[Preguntak][ind_v2] = df_PGA["Respuesta"][ind_v1[0]]
        df_PGA_v2[Preguntak].replace(np.nan,0)
        df_PGA_v2["TOTAL"] = df_PGA_v2["TOTAL"] + df_PGA_v2[Preguntak]
df_PGA_v2.head()
            
df_PGA_v2.TOTAL = round(df_PGA_v2.TOTAL/3,0)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Hay inconsistencias entre el Total reportado en la tabla original y el calculado a partir de la calificación de cada uña. Esto se observa también en EXCEL.

In [5]:
del df_PGA_v2["Total"]
del df_PGA_v2["Respuesta"]

df_PGA_v2["Ano"]=pd.DatetimeIndex(df_PGA_v2["Fecha"]).year
df_PGA_v2["Mes"]=pd.DatetimeIndex(df_PGA_v2["Fecha"]).month
df_PGA_v2["Dia"]=pd.DatetimeIndex(df_PGA_v2["Fecha"]).day

df_PGA_v2

,ID,Fecha,TOTAL,Eritema,Endurecimiento,Descamación,Ano,Mes,Dia
0,502369.0,2019-04-24 07:54:55,0.0,0.0,0.0,0.0,2019,4,24
1,502369.0,2019-09-16 13:38:46,2.0,3.0,3.0,1.0,2019,9,16
2,502369.0,2020-01-29 09:03:42,0.0,0.0,0.0,0.0,2020,1,29
3,506165.0,2019-09-16 13:44:15,1.0,1.0,1.0,1.0,2019,9,16
4,510859.0,2019-08-12 14:28:20,3.0,3.0,3.0,4.0,2019,8,12
...,...,...,...,...,...,...,...,...,...
2834,3499636.0,2019-07-05 09:08:19,0.0,0.0,0.0,0.0,2019,7,5
2835,3499636.0,2019-10-30 13:59:15,0.0,0.0,0.0,0.0,2019,10,30
2836,3499636.0,2020-02-26 11:26:13,0.0,0.0,0.0,0.0,2020,2,26
2837,3499725.0,2019-05-24 12:48:58,0.0,0.0,0.0,0.0,2019,5,24


In [6]:
import unicodedata
def rem_accents(text):
    return unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")


In [7]:
columns = df_PGA_v2.columns
new_columns = {}
for i in columns:
    new_columns[i] = rem_accents(i).lower().replace(" ","_")

for i in columns:
    df_PGA_v2[i] = df_PGA_v2[i].apply(lambda x: rem_accents(str(x).lower()))
    
df_PGA_v2.rename(columns=new_columns, inplace=True)
df_PGA_v2.head()

,id,fecha,total,eritema,endurecimiento,descamacion,ano,mes,dia
0,502369.0,2019-04-24 07:54:55,0.0,0.0,0.0,0.0,2019,4,24
1,502369.0,2019-09-16 13:38:46,2.0,3.0,3.0,1.0,2019,9,16
2,502369.0,2020-01-29 09:03:42,0.0,0.0,0.0,0.0,2020,1,29
3,506165.0,2019-09-16 13:44:15,1.0,1.0,1.0,1.0,2019,9,16
4,510859.0,2019-08-12 14:28:20,3.0,3.0,3.0,4.0,2019,8,12


In [8]:
df_PGA_v2.to_csv("PGA_DF.csv", index = False)